# 🤺 Demo: Basic usage

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. 

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `Template` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [1]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

In [2]:
%load_ext autoreload
from fence.models.bedrock.claude import Claude35Sonnet
from fence.models.openai import GPT4omini
from fence.templates.string import StringTemplate
from fence.templates.messages import MessagesTemplate, Messages, Message
from fence.links import Link, TransformationLink
from fence.chains import Chain, LinearChain

## ⚙️ Setting up

In [3]:
# Get our model
model = Claude35Sonnet(source='demo_notebook', region='us-east-1')
model = GPT4omini(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [4]:
# Use the invoke method to call the model
model.invoke('Why is the sky blue?')

"The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it is made up of different colors, which correspond to different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, it collides with air molecules and small particles. The shorter wavelengths of blue light are scattered in all directions more effectively than the longer wavelengths of red light. This scattering causes blue light to dominate the colors that we see when we look up at the sky.\n\nDuring sunrise and sunset, the sun's light has to travel through a thicker layer of the atmosphere, which scatters the shorter blue wavelengths out of direct sight and allows the longer wavelengths (reds and oranges) to be more prominent. This is why we often see beautiful colors during those times."

In [5]:
# Just call the damn thing
model('Why is the sky blue?')

"The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it interacts with molecules and small particles in the air. Sunlight, or white light, is made up of many different colors, each with a different wavelength. \n\nBlue light has a shorter wavelength compared to other colors like red or yellow. Because shorter wavelengths scatter more than longer wavelengths, the blue light is scattered in all directions by the oxygen and nitrogen molecules in the atmosphere. This scattering causes the sky to look predominantly blue to our eyes.\n\nDuring sunrise and sunset, the sun's light passes through a thicker layer of the atmosphere. The longer path results in even more scattering of the shorter wavelengths, allowing the longer wavelengths (reds and oranges) to dominate, giving the sky a reddish hue during those times."

### 🔨 Level 2 - Use a PromptTemplate

In [6]:
# Initialize a prompt template
prompt_template = StringTemplate('Why is the sky {color}?')
print(prompt_template)

StringTemplate: Why is the sky {color}?


In [7]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky red?


In [8]:
prompt_template_test = StringTemplate('Why is the sky {color}?')
print(prompt_template_test.render({'color': 'blue'}))

Why is the sky blue?


In [9]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = StringTemplate('Why is the sky {color}?')
prompt_template_grass = StringTemplate('Why is the grass {color}?')
prompt_template_dress = StringTemplate('I like a dress with {pattern}.')
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

StringTemplate: Why is the sky {color}? Why is the grass {color}? I like a dress with {pattern}.
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [10]:
# You can customize the separator
base_template = StringTemplate('Why is the sky {color}?', separator=' FUNKY TOWN ')
additional_template = StringTemplate('Why is the grass {color}?')
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


In [11]:
# You can also use the MessagesTemplates for more complex prompts
messages = Messages(
    system='Respond in a {tone} tone',
    messages= [
        Message(role="user", content="Why is the sky {color}?"),
        # Equivalent to Message(role='user', content=Content(type='text', text='Why is the sky blue?'))
        # But Content can also be an image, etc.
    ]
)
messages_template = MessagesTemplate(
    source=messages
)
print(messages_template)
print(f"\nRendered: {messages_template.render(tone='sarcastic', color='blue')}")

MessagesTemplate: messages=[Message(role='user', content='Why is the sky {color}?')] system='Respond in a {tone} tone'

Rendered: messages=[Message(role='user', content='Why is the sky blue?')] system='Respond in a sarcastic tone'


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [12]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, model=model)
print(link)

Link: <['color']> -> <state>


In [13]:
# Invoke it
link(color='blue') # Or, equivalently, link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. This occurs when sunlight, which is made up of many colors (or wavelengths), interacts with the Earth's atmosphere.\n\nSunlight contains different colors that correspond to different wavelengths. Blue light has a shorter wavelength compared to other colors in the spectrum, such as red and yellow. When sunlight passes through the atmosphere, it collides with gas molecules and small particles. The shorter wavelengths of light (blue and violet) are scattered more than the longer wavelengths (red and yellow).\n\nEven though violet light is scattered even more than blue, our eyes are more sensitive to blue light and the upper atmosphere absorbs some violet light, which is why we predominantly see a blue sky during the day. When the sun is lower on the horizon, such as during sunrise and sunset, the light path through the atmosphere is longer, allowing more scattering of the shorter wavelengths and leav

In [14]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, model=model, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it interacts with air molecules and small particles. Sunlight, or white light, is made up of various colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nBecause of this shorter wavelength, blue light is scattered in all directions by the gases and particles in the atmosphere. As a result, when we look up at the sky during the day, we see more of this scattered blue light than any other color. \n\nDuring sunrise and sunset, the sky can appear red or orange because the sunlight passes through a thicker portion of the atmosphere. This causes shorter wavelengths (blue and green light) to scatter out more, allowing the longer wavelengths (red and orange) to dominate the view.",
 'intermediate': "The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's 

In [15]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, model=model, name='sky')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue due to a phenomenon known as Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red and yellow.\n\nAs sunlight passes through the atmosphere, it interacts with air molecules and small particles. Because blue light is scattered in all directions more effectively than other colors due to its shorter wavelength, it becomes the dominant color we see when we look up at the sky. \n\nDuring sunrise and sunset, the sunlight passes through a greater thickness of the Earth's atmosphere, scattering shorter wavelengths of light and allowing the longer wavelengths (reds and oranges) to dominate, which is why the sky can appear red or orange during those times."}

In [16]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [17]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=StringTemplate(
        "What's the opposite of {A}? Reply with a few words max."
    ),
    name = 'opposite',
    output_key="X",
)
link_synonym = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word."
    ),
    name='synonym',
    output_key="Y",
)
link_poem = Link(
    template=StringTemplate(
        "Write a poem about {state}. Return only the poem, beginning with the title."
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(model=model, links=[link_opposite, link_synonym, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
from pprint import pprint
pprint(result)

{'A': 'A police officer',
 'B': 'Hopeful',
 'C': 'A criminal. and Optimistic.',
 'X': 'A criminal.',
 'Y': 'Optimistic.',
 'Z': "**The Optimist's Crime**\n"
      '\n'
      'In shadows deep where silence hides,  \n'
      'A criminal’s heart still beats, it bides,  \n'
      'With dreams that sparkle like stars above,  \n'
      'Whispers of hope, a longing for love.  \n'
      '\n'
      'He prowls the streets with a crooked smile,  \n'
      'Yet in his chest, a spark worthwhile,  \n'
      'For every heist, a tale of grace,  \n'
      'A wish for redemption, a chance to embrace.  \n'
      '\n'
      'With fingers stained by the sins of night,  \n'
      'He counts his fortunes with fingers tight,  \n'
      'But in his eyes, a glimmering thread,  \n'
      'Of laughter, of joy, of paths ahead.  \n'
      '\n'
      'He dreams of a life where wrong becomes right,  \n'
      'Where burdens lift softly like clouds taking flight,  \n'
      'Each stolen moment a lesson learned,  \n'
 

In [18]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=StringTemplate(
        "Capitalize this word: {A}. Only respond with the capitalized version",
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word.", 
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=StringTemplate(
        "Combine {X} and {Y} and {C} in a meaningful sentence.", 
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(model=model, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['C', 'X', 'Y']> -> <Z>]

In [19]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'A', 'C', 'B'}. Missing: {'C'}


In [20]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

The optimistic police officer, accompanied by his loyal dog, patrolled the neighborhood with a smile, spreading positivity and ensuring the safety of the community.


In [21]:
# Cycles are not allowed
link_up = Link(
    template=StringTemplate(
        "Capitalize this word: {up}. Only respond with the capitalized version",
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {down}. Reply with one word.", 
    ),
    name='down',
    output_key="up",
)
chain = Chain(model=model, links=[link_up, link_b])
try:
    chain.run(input_dict={"up": "happy"})
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
